## <p style="text-align:center">Bond Key Rate Durations calculation in FinancePy</p>


In [1]:
from financepy.utils.date import Date
from financepy.utils.day_count import DayCountTypes
from financepy.utils.frequency import FrequencyTypes
from financepy.products.bonds import *
from financepy.market.curves import *


####################################################################
# FINANCEPY BETA Version 0.280 - This build:  23 Apr 2023 at 16:02 #
#     This software is distributed FREE AND WITHOUT ANY WARRANTY   #
#  Report bugs as issues at https://github.com/domokane/FinancePy  #
####################################################################



Modified duration measures a bond's overall price sensitivity to interest rate changes, while key rate durations measure sensitivity at specific points on the yield curve. 

In [2]:
# First, let's create a bond object.

accrual_type, frequencyType, settlementDays, exDiv, calendar = get_bond_market_conventions(
    BondMarkets.UNITED_STATES)

# interest accrues on this date. Original issue date is 01/08/2022
issue_date = Date(31, 7, 2022)
maturity_date = Date(31, 7, 2027)
cpn = 2.75/100
bond = Bond(issue_date, maturity_date, cpn, frequencyType, accrual_type)
settlement_date = Date(24, 4, 2023)  # next settle date for this bond
bond


OBJECT TYPE: Bond
ISSUE DATE: 31-JUL-2022
MATURITY DATE: 31-JUL-2027
COUPON (%): 2.75
FREQUENCY: FrequencyTypes.SEMI_ANNUAL
ACCRUAL TYPE: DayCountTypes.ACT_ACT_ICMA
FACE AMOUNT: 100.0

In [19]:
# the key_rate_durations function returns a tuple of key_rate_tenors, key_rate_durations

# US Street yield on Bloomberg as of 20 April 2023
# with settle date 24 April 2023
ytm = 3.725060/100

# settlement date is the date on which the bond is traded or valued

tenors, krds = bond.key_rate_durations(settlement_date, ytm)

In [20]:
for tenor, krd in zip(tenors, krds):
    print(tenor, ">>>", round(krd,6))   

0.5 >>> -0.000957
1.0 >>> -0.009298
2.0 >>> -0.021571
3.0 >>> 1.435719
5.0 >>> 2.547636
7.0 >>> 0.0
10.0 >>> 0.0
20.0 >>> 0.0
30.0 >>> 0.0


In [21]:
# Modified duration of a bond and sum of the key rate duration should be close to each other.

mod_dur = bond.modified_duration(settlement_date, ytm)
sum_of_krds = sum(krds)

print("Modified duration ", mod_dur)
print("Sum of key rate durations ", sum_of_krds)

Modified duration  3.9544603137385024
Sum of key rate durations  3.9515292410905456


<p style="text-align:right">prepared by github.com/sagayev </p>

Below is a screenshot from Bloomberg for this bonds key rate durations. For reference, ISIN is US91282CFB28, and Cusip is 91282CFB2 as of 20 April 2023
Please note that, Key rate durations analytics on Bloomberg standard terminal are available only for last data. So, if you want to replicate, you can view it's last data and change the settlement_date above.

Key rate durations method of the Bond class takes two mandatory parameters: settlement_date (or valuation date) and Yield to maturity. Optionally, we can provide our own key rate tenors (in years) and our own shift value. 
Shift value defaults to 0.0001 and is used to shift the key rate of a corresponding tenor to assess the change in bond price.
Default key rate tenors are 0.5, 1, 2, 3, 5, 7, 10, 20, 30

![Alt text](data/Key_rate_duration_US91282CFB28.png)

If we want to approximate above key rate duration numbers, we have to use the yield curve data as of 20 April 2023

![Alt text](data/Yield_curve_data_for_krds.png)

In [13]:
my_tenors = np.array([0.5,  1,  2,  3,  5,  7,  10])

my_rates = np.array([5.0367, 4.7327, 4.1445, 3.8575, 3.6272,  3.5825,  3.5347])/100

tenors, krds = bond.key_rate_durations(settlement_date, ytm, key_rate_tenors= my_tenors, rates= my_rates)
for tenor, krd in zip(tenors, krds):
    print(tenor, ">>>", round(krd,6))  

0.5 >>> -0.000948
1.0 >>> -0.009258
2.0 >>> -0.021624
3.0 >>> 1.432306
5.0 >>> 2.527236
7.0 >>> 0.0
10.0 >>> 0.0


In [14]:
tenors, krds = bond.key_rate_durations(settlement_date, ytm, key_rate_tenors= my_tenors, rates= my_rates)


In [15]:
tenors

array([ 0.5,  1. ,  2. ,  3. ,  5. ,  7. , 10. ])

In [16]:
krds

array([-9.26677041e-04, -9.17865435e-03, -2.14366404e-02,  1.43250552e+00,
        2.52700338e+00,  0.00000000e+00,  0.00000000e+00])